### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [46]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [47]:
import csv
import doctest

def calculate_bmi(weight_kg, height_m):
    """
    Calculate BMI given weight in kg and height in meters.

    Args:
    weight_kg (float): The weight of the patient in kilograms.
    height_m (float): The height of the patient in meters.

    Returns:
    float: The calculated BMI value.

    Raises:
    ValueError: If height is less than or equal to zero.

    Examples:
    >>> calculate_bmi(70, 1.75)
    22.86
    >>> calculate_bmi(90, 1.8)
    27.78
    >>> calculate_bmi(50, 1.6)
    19.53
    >>> calculate_bmi(80, 0)
    Traceback (most recent call last):
    ...
    ValueError: Height must be greater than zero
    """
    if height_m <= 0:
        raise ValueError("Height must be greater than zero")

    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 2)

if __name__ == "__main__":
    doctest.testmod()


### Step 2: Calculate BODE Score

In [49]:
def calculate_bode_score(fev_pct, dyspnea_description, distance_meters, bmi):
    """
    Calculates the total BODE score.

    Args:
        fev_pct (float): The FEV1 percentage predicted.
        dyspnea_description (str): The dyspnea description.
        distance_meters (float): The distance walked in 6 minutes.
        bmi (float): The Body Mass Index.

    Returns:
        int: The total BODE score.
    """
    # Calculate FEV1 score
    fev_score = 0
    if fev_pct >= 65:
        fev_score = 0
    elif 50 <= fev_pct < 65:
        fev_score = 1
    elif 36 <= fev_pct < 50:
        fev_score = 2
    else:
        fev_score = 3

    # Calculate BMI score
    bmi_score = 1 if bmi < 21 else 0

    # Map dyspnea description to score
    dyspnea_score_map = {
        "WHEN HURRYING": 1,
        "STOPS AFTER A FEW MINUTES": 2,
        "BREATHLESS WHEN DRESSING": 3
    }
    dyspnea_score = dyspnea_score_map.get(dyspnea_description.upper(), 0)

    # Calculate distance walked score
    if distance_meters >= 350:
        distance_score = 0
    elif 250 <= distance_meters < 350:
        distance_score = 1
    elif 150 <= distance_meters < 250:
        distance_score = 2
    else:
        distance_score = 3

    # Total BODE score
    total_bode_score = fev_score + bmi_score + dyspnea_score + distance_score
    return total_bode_score


### Step 3: Calculate BODE Risk

In [50]:
def calculate_bode_risk(bode_score):
    """
    Calculate the BODE risk based on the BODE score.

    Args:
    bode_score (int): The BODE score (0 to 10).

    Returns:
    float: The BODE risk as a percentage.

    Raises:
    ValueError: If the BODE score is out of range.

    Examples:
    >>> calculate_bode_risk(2)
    90
    >>> calculate_bode_risk(4)
    75
    >>> calculate_bode_risk(6)
    50
    >>> calculate_bode_risk(8)
    30
    >>> calculate_bode_risk(11)
    Traceback (most recent call last):
    ...
    ValueError: BODE score must be between 0 and 10
    """
    if bode_score < 0 or bode_score > 10:
        raise ValueError("BODE score must be between 0 and 10")

    if 0 <= bode_score <= 2:
        return 90  # Low risk, high survival
    elif 3 <= bode_score <= 4:
        return 75  # Moderate risk
    elif 5 <= bode_score <= 6:
        return 50  # High risk
    elif 7 <= bode_score <= 10:
        return 30  # Very high risk (low survival)

if __name__ == "__main__":
    doctest.testmod()


### Step 4: Load Hospital Data

In [54]:
import json

def load_hospital_data(json_file_path):
    """
    Load hospital data from a JSON file.

    Args:
    json_file_path (str): The path to the JSON file containing hospital data.

    Returns:
    dict: A dictionary where the key is the hospital name and the value is the number of beds.

    Raises:
    FileNotFoundError: If the JSON file does not exist.
    ValueError: If the JSON file has invalid structure or content.

    Examples:
    >>> hospitals = load_hospital_data("/content/hospitals (1).json")
    >>> hospitals["BJC"]
    2000
    >>> hospitals["ST.LUKE'S"]
    800
    """
    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        hospital_dict = {}

        for system in data:
            for hospital in system['hospitals']:
                hospital_dict[hospital['name']] = hospital['beds']

        return hospital_dict

    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {json_file_path}")
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON file structure or content")

# Example usage with the correct file path
if __name__ == "__main__":
    # Use the correct path based on your system
    json_file_path = "/content/hospitals (1).json"
    hospitals = load_hospital_data(json_file_path)
    print(hospitals)


{'BJC': 2000, 'BJC WEST COUNTY': 1000, 'MISSOURI BAPTIST': 800, 'SAINT LOUIS UNIVERSITY': 1000, "ST.MARY'S": 500, "ST.LUKE'S": 800}


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [61]:
import os
import csv
import json

# Ensure output directory exists
output_directory = './output/'  # Use local directory if necessary
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
print(f"Directory created or exists: {output_directory}")

# Set file paths
patient_file_path = './patient.csv'  # Adjust this to a valid path for patient file
hospital_file_path = './hospitals.json'  # Adjust this to a valid path for hospital file
patient_output_file = os.path.join(output_directory, 'patient_output.csv')
hospital_output_file = os.path.join(output_directory, 'hospital_output.csv')

print(f"Writing patient output to {patient_output_file}")
print(f"Writing hospital output to {hospital_output_file}")

# Function to process files (use the previous provided code for calculations)

Directory created or exists: ./output/
Writing patient output to ./output/patient_output.csv
Writing hospital output to ./output/hospital_output.csv
